In [1]:
!pip install transformers
!pip install pandas
!pip install torch

In [2]:

from huggingface_hub import notebook_login
import pandas as pd
from transformers import LlamaTokenizer, LlamaForCausalLM, AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import transformers
from tqdm import tqdm
from torch.utils.data import Dataset

In [3]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import tqdm

# Check if GPU is available and set the device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")
token = ""  # Replace with your token

# Load the CSV dataset
df = pd.read_csv('DiseasePredictor_test.csv')

# Load the model and tokenizer
model_name = "meta-llama/Llama-3.1-8B-Instruct"  # Use the same model as before

model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=token, torch_dtype=torch.float16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=token)

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Define the query_model function to generate short answers
def query_model(prompt, temperature=0.0, max_length=50):
    prompt = f"Given the symptoms of the patient, choose the most likely disease from the options below. Respond with the disease name only.\n\n{prompt}\nAnswer:"

    sequences = pipeline(
        prompt,
        do_sample=False,  # No randomness, for deterministic answers
        temperature=temperature,  # No randomness for deterministic output
        max_new_tokens=max_length,  # Limit output length to 50 tokens
        return_full_text=False,
        pad_token_id=tokenizer.eos_token_id
    )

    answer = sequences[0]['generated_text'].strip()
    return answer

# Initialize an empty list to store the generated answers
generated_answers = []

# Loop through each row in the dataset
for index, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    try:
        question = row['Question']
        # Query the model with the current question
        response = query_model(question, temperature=0.0)
        print(response)
        generated_answers.append(response)
    except Exception as e:
        print(f"An error occurred for row {index}: {e}")
        generated_answers.append(None)  # Append None for failed generations

# Add the generated answers to the dataframe
df['Answer'] = generated_answers

# Save the dataframe with the answers to a new Excel file
df.to_csv('DiseaseDecipher_with_Answers.csv', index=False)

print("Model answers have been saved to 'DiseaseDecipher_with_Answers.csv'.")


Using GPU


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

2) Otosclerosis
Explanation: Otosclerosis is a condition in which abnormal bone growth in the middle ear causes hearing loss. The symptoms of otosclerosis include hearing loss, ear pain, and a plugged feeling in the ear. The


100%|█████████▉| 3199/3200 [3:04:18<00:03,  3.46s/it]

3) Otosclerosis
Explanation: Otosclerosis is a condition in which abnormal bone growth in the middle ear causes hearing loss. The symptoms of otosclerosis include hearing loss, ear pain, and a plugged feeling in the ear. The


100%|██████████| 3200/3200 [3:04:21<00:00,  3.46s/it]

4) Otosclerosis
Explanation: Otosclerosis is a condition in which abnormal bone growth in the middle ear causes hearing loss. The symptoms of otosclerosis include hearing loss, ear pain, and a plugged feeling in the ear. The
Model answers have been saved to 'DiseaseDecipher_with_Answers.csv'.
